# Propensity score matching

A propensity score is the probability that a unit with certain characteristics will be assigned to the treatment group (as opposed to the control group). The scores can be used to **reduce or eliminate selection bias in observational studies by balancing covariates** (the characteristics of participants) between treated and control groups. When the covariates are balanced, it become much easier to match participants with multiple characteristics.

Propensity score matching creates sets of participants for treatment and control groups. A matched set consists of at least one participant in the treatment group and one in the control group with similar propensity scores. The goal is to approximate a random experiment, eliminating many of the problems that come with observational data analysis


## Basic Steps

The basic steps to propensity score matching are:

- Collect and prepare the data.

- Estimate the propensity scores. The true scores are unknown, but can be estimated by many methods including: discriminant analysis, logistic regression, and random forests. The “best” method is up for debate, but one of the more popular methods is logistic regression.

- Match the participants using the estimated scores.

- Evaluate the covariates for an even spread across groups. The scores are good estimates for true propensity scores if the matching process successfully distributes covariates over the treated/untreated groups (Ho et. al, 2007).

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from pymatch.Matcher import Matcher #Match data for an observational study.
import pandas as pd
import numpy as np
import random

In [3]:
df=pd.read_csv('casual inference/data/trainees.csv')
control = df[df['trainees']==0]
treat = df[df['trainees']==1]

In [4]:
control['earnings'].mean()-treat['earnings'].mean()

4297.49373433584

If I do a simple comparison in means, we get that the trainees earn less money than those that didn't go through the program. However, if we look at the table above, we notice that trainees are much younger than non trainees, which indicates that **age is probably a confounder**. 

Let's use matching on age to try to correct that. We will take unit 1 from the treated and pair it with unit 27, since both are 28 years old. Unit 2 we will pair it with unit 34, unit 3 with unit 37, unit 4 we will pair it with unit 35... When it comes to unit 5, we need to find someone with age 29 from the non treated, but that is unit 37, which is already paired. This is not a problem, since **we can use the same unit multiple times. If more than 1 unit is a match, we can choose randomly between them.**

This is what the matched dataset looks like for the first 7 units

In [10]:
unique_on_age=control.drop_duplicates('age')
matches=treat.merge(unique_on_age,on='age',how='left',suffixes=("_t_1", "_t_0")
           ).assign(diff = lambda d: d["earnings_t_1"] - d["earnings_t_0"])
matches.head(3)

,unit_t_1,trainees_t_1,age,earnings_t_1,unit_t_0,trainees_t_0,earnings_t_0,diff
0,1,1,28,17700,27,0,8800,8900
1,2,1,34,10200,34,0,24200,-14000
2,3,1,29,14400,37,0,6200,8200


In [11]:
matches["diff"].mean()

2457.8947368421054


Create test and control groups and reassign loan_status to be a binary treatment indicator. This is our reponse in the logistic regression model(s) used to generate propensity scores.

But this was a very contrived example, just to introduce matching. In reality, we usually have more than one feature and units don't match perfectly. In this case, **we have to define some measurement of proximity to compare how units are close to each other.** 

One common metric for this is the euclidean norm $||X_i - X_j||$. This difference, however, is not invariant to the scale of the features. This means that features like age, that take values on the tenths, will be much less important when computing this norm compared to features like income, which take the order of hundreds. For this reason, before applying the norm, we need to scale the features so that they are on roughly the same scale.

Having defined a distance measure, we can now define the match as the nearest neighbour to that sample we wish to match. 

To test this estimator, let's consider a medicine example. Once again, we want to find the effect of a medication on days until recovery. Unfortunately, this effect is confounded by severity, sex and age. We have reasons to believe that patients with more severe conditions have a higher chance of receiving the medicine.

In [12]:
med = pd.read_csv("casual inference/data/medicine_impact_recovery.csv")
med.head()

,sex,age,severity,medication,recovery
0,0,35.049134,0.887658,1,31
1,1,41.580323,0.899784,1,49
2,1,28.127491,0.486349,0,38
3,1,36.375033,0.323091,0,35
4,0,25.091717,0.209006,0,15


In [13]:
med.query("medication==1")["recovery"].mean() - med.query("medication==0")["recovery"].mean()

16.895799546498726

If we look at a simple difference in means, $E[Y|T=1]-E[Y|T=0]$, we get that the treatment takes, on average, 16.9 more days to recover than the untreated. This is probably due to confounding, since we don't expect the medicine to cause harm to the patient.

To correct for this bias, we will control for features using matching. First, we need to scale our features, otherwise, features like age will have higher importance than features like severity when we compute the distance between points. To do so, we can standardise the features.

In [18]:
#standardise the features
from sklearn.preprocessing import StandardScaler
scaling = StandardScaler()
med[['sex']]=scaling.fit_transform(med[['sex']])
med[['age']]=scaling.fit_transform(med[['age']])
med[['severity']]=scaling.fit_transform(med[['severity']])

In [15]:
med.head()

,sex,age,severity,medication,recovery
0,0,35.049134,0.887658,1,31
1,1,41.580323,0.899784,1,49
2,1,28.127491,0.486349,0,38
3,1,36.375033,0.323091,0,35
4,0,25.091717,0.209006,0,15


Now, to the matching itself. Instead of coding a matching function, we will use the K nearest neighbour algorithm. This algorithm makes predictions by finding the nearest data point in an estimation or training set.

For matching, we will need 2 of those. One, mt0, will store the untreated points and will find matches in the untreated when asked to do so. The other, mt1, will store the treated point and will find matches in the treated when asked to do so. After this fitting step, we can use these KNN models to make predictions, which will be our matches.

In [16]:
from sklearn.neighbors import KNeighborsRegressor
treat=med[med['medication']==1]
control=med[med['medication']==0]
X=['sex','age','severity']
y=['recovery']
knn= KNeighborsRegressor(n_neighbors=1)
knn_0= knn.fit(control[X], control[y])
knn_1= knn.fit(treat[X], treat[y])

In [20]:
# find matches for the treatment group looking at the control knn model
match_treat=knn_0.predict(treat[X])
# find matches for the control looking at the treatment knn model
match_control=knn_1.predict(control[X])

predicted = pd.concat([treat.assign(match=match_treat),control.assign(match=match_control)])
predicted.head(2)

,sex,age,severity,medication,recovery,match
0,0,35.049134,0.887658,1,31,31.0
1,1,41.580323,0.899784,1,49,49.0


In [21]:
np.mean((2*predicted["medication"] - 1)*(predicted["recovery"] - predicted["match"]))

3.02715

Using this sort of matching, we can see that the effect of the medicine is still positive anymore. This means that, controlling for X, the medicine increase the recovery time by about 1 day, on average. This is already a huge improvement on top of the biased estimate that predicted a 16.9 increase in recovery time.

However, we can still do better.

In [ ]:
from sklearn.linear_model import LinearRegression
lr= LinearRegression()
lr_0=lr.fit(control[X], control[y])
lr_1=lr.fit(treat[X], treat[y])

# find the units that match to the treated
treated_match_index = knn_0.kneighbors(treat[X], n_neighbors=1)[1].ravel()
# find the units that match to the untreatd
control_match_index = knn_1.kneighbors(control[X], n_neighbors=1)[1].ravel() # ravel() change a 2-dimensional array or a multi-dimensional array into a contiguous flattened array. 

predicted = pd.concat([
    (treat
     # find the Y match on the other group
     .assign(match=knn_0.predict(treat[X])) 
     
     # build the bias correction term
     .assign(bias_correct=lr_0.predict(treat[X]) - lr_0.predict(control.iloc[treated_match_index][X]))),
    (control
     .assign(match=knn_1.predict(control[X]))
     .assign(bias_correct=lr_1.predict(control[X]) - lr_0.predict(treat.iloc[control_match_index][X])))
])
predicted.head()

First of all, this linear regression that we are fitting doesn't extrapolate on the treatment dimension to get the treatment effect. Instead, its purpose is just to correct bias. Linear regression here is local, in the sense that it doesn't try to see how the treated would be if it looked like the untread. It does none of that extrapolation. This is left to the matching part. The meat of the estimator is still the matching component. The point I want to make here is that OLS is secondary to this estimator.

The second point is that matching is a non-parametric estimator. It doesn't assume linearity or any kind of parametric model. As such, it is more flexible than linear regression and can work in situations where linear regression will not, namely, those where non linearity is very strong.

With the bias correction formula, I get the following ATE estimation.

In [ ]:
np.mean((2*predicted["medication"] - 1)*((predicted["recovery"] - predicted["match"])-predicted["bias_correct"]))

In [ ]:
# conduct a confidence interval 
from causalinference import CausalModel
cm = CausalModel(
    Y=med["recovery"].values, 
    D=med["medication"].values, 
    X=med[["severity", "age", "sex"]].values
)

cm.est_via_matching(matches=1, bias_adj=True)
print(cm.estimates)

Finally, we can say with confidence that our medicine does indeed lower the time someone spends at the hospital.